<a href="https://colab.research.google.com/github/bgalerne/IoT_mathematics/blob/master/Lab3_eigendecomposition_and_svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVD decomposition for background removal in video

Adapted form:
https://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra-v2/blob/master/nbs/02-Background-Removal-with-SVD.ipynb#Load-and-Format-the-Data


In [ ]:
import imageio
imageio.plugins.ffmpeg.download()

In [ ]:
import moviepy.editor as mpe
import numpy as np
import scipy
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
np.set_printoptions(precision=4, suppress=True)

# Download video data:

Full video can be seen here (middle screen):
https://www.youtube.com/watch?v=VcJIpf_X-iA

Source: http://jacarini.dinf.usherbrooke.ca/dataset2012/

In [ ]:
!wget http://jacarini.dinf.usherbrooke.ca/static/dataset/intermittentObjectMotion/sofa.zip


In [ ]:
!unzip sofa.zip

In [ ]:
A = imageio.imread('sofa/input/in000001.jpg')[1::3,1::3,1]
h, w = A.shape
Npix = h*w
A = A.ravel()
#G = A[:,:,1]
#print(G.shape)
B = imageio.imread('sofa/input/in000002.jpg')[:,:,1]

Nframe = 2750
M = np.zeros((Npix,Nframe))


for i in tqdm(range(Nframe-1)):
  str='sofa/input/in%06d.jpg' % (i+2)
  M[:,i-1]=imageio.imread(str)[1::3,1::3,1].ravel()




**Question:** Explain each step.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
plt.imshow(M,cmap='gray')
print(M.shape)

# Compute SVD of spatial x times data:

In [ ]:
U, s, V = np.linalg.svd(M, full_matrices=False)



In [ ]:
# save on disc
np.save("U.npy", U)
np.save("s.npy", s)
np.save("V.npy", V)

In [ ]:
# #load on disc:
# U = np.load("U.npy")
# s = np.load("s.npy")
# V = np.load("V.npy")

In [ ]:
print(U.shape)
print(s.shape)
print(V.shape)

In [ ]:
RM = U @ np.diag(s) @ V
np.allclose(M, RM)

In [ ]:
np.set_printoptions(suppress=True, precision=0)

In [ ]:
s[0::50]

In [ ]:
low_rank = np.expand_dims(U[:,0], 1) * s[0] * np.expand_dims(V[0,:], 0)

In [ ]:
plt.imshow(np.reshape(low_rank[:,0], (h,w)), cmap='gray');

# Display frame VS frame minus background

In [ ]:
fnb = np.arange(0,Nframe,300) # frame number
print(fnb)
fig, ax = plt.subplots(len(fnb),2,figsize=(20*len(fnb), 20*len(fnb)))
for i in range(len(fnb)):
  ax[i,0].imshow(np.reshape(M[:,fnb[i]], (h,w)), cmap='gray');
  ax[i,1].imshow(np.reshape(np.abs(M[:,fnb[i]] - low_rank[:,fnb[i]]), (h,w)), cmap='gray');

# Make an extract movie

In [ ]:
from moviepy.video.io.bindings import mplfig_to_npimage

In [ ]:
def make_video(matrix, dims, filename):
    mat_reshaped = np.reshape(matrix, (dims[0], dims[1], -1))
    
    fig, ax = plt.subplots()
    def make_frame(t):
        ax.clear()
        ax.imshow(mat_reshaped[...,int(t*fps)])
        return mplfig_to_npimage(fig)
    
    animation = mpe.VideoClip(make_frame, duration=int(10))
    animation.write_videofile(filename + '.mp4', fps=fps)

In [ ]:
dims = (h,w)
fps = 30
make_video(np.abs(M - low_rank), dims, "figures2")

In [ ]:
mpe.VideoFileClip("figures2.mp4").ipython_display(width=300)